In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

In [2]:
data = pd.read_csv("dataset\Coursera.csv")
data

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...
...,...,...,...,...,...,...,...
3517,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,https://www.coursera.org/learn/python-data-vis...,"In the capstone, students will build a series ...",Databases syntax analysis web Data Visuali...
3518,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,https://www.coursera.org/learn/henry,"�Give me liberty, or give me death:� Rememberi...",retirement Causality career history of the ...
3519,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,https://www.coursera.org/learn/business-intell...,�Megatrends� heavily influence today�s organis...,analytics tableau software Business Intellig...
3520,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/rigid-body-dyna...,"This course teaches dynamics, one of the basic...",Angular Mechanical Design fluid mechanics F...


In [3]:
data.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

In [4]:
data=data[['Course Name','Difficulty Level','Course Description','Skills']]

In [5]:
def remove_punctuation(txt):
  txt_nopunt= "".join([c for c in txt if c not in string.punctuation])
  return txt_nopunt

In [6]:
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Abrar
[nltk_data]     Shah\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [7]:
data['Course Description'] = data['Course Description'].str.replace('http\S+|www.\S+', '', case=False)

In [8]:
data['Course Description']=data['Course Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data['Course Name']=data['Course Name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data['Skills']=data['Skills'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

data['Course Name']=data['Course Name'].replace(","," ",regex=True)
data['Course Description']=data['Course Description'].replace(","," ",regex=True)
data['Skills']=data['Skills'].replace(","," ",regex=True)
data['Skills']=data['Skills'].replace("-"," ",regex=True)

data['Course Name']=data['Course Name'].apply(lambda x: remove_punctuation(x))
data['Course Description']=data['Course Description'].apply(lambda x: remove_punctuation(x))
data['Skills']=data['Skills'].apply(lambda x: remove_punctuation(x))

data['Course Description']=data['Course Description'].replace("�","'",regex=True)

data['tags']=data['Course Name']+' '+ data['Difficulty Level']+' '+ data['Course Description']+' '+data['Skills']
new_data=data[['Course Name','tags']]

In [9]:
new_data['tags'] = new_data['tags'].apply(lambda x:x.lower())

In [10]:
new_data['tags']=new_data['tags'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [11]:
new_data['tags'][0]

'write feature length screenplay film television beginner write full length feature film script course write complete featurelength screenplay film television serious drama romantic comedy anything learn break creative process components discover structured process allows produce polished pitchready script end course completing project increase confidence ideas abilities feel prepared pitch first script get started next course designed tap creativity based active learning actual learning takes place within activities writing learn link trailer course view trailer please copy paste link browser httpsvimeocom382067900b78b800dc0 learner review love approach professor wheeler takes towards course point easy follow informative would definitely recommend anyone interested taking screenplay writing course course curriculum simple adopt professional writers room process write post work peer review share feedback peers revise work feedback receive peers thats real world feel professional writer

In [12]:
ps=PorterStemmer()

In [13]:
def stem(text):
  y=[]
  
  for i in text.split():
    y.append(ps.stem(i))
  
  return " ".join(y)

In [14]:
new_data['tags']=new_data['tags'].apply(stem)

In [15]:
new_data['tags'][0]

'write featur length screenplay film televis beginn write full length featur film script cours write complet featurelength screenplay film televis seriou drama romant comedi anyth learn break creativ process compon discov structur process allow produc polish pitchreadi script end cours complet project increas confid idea abil feel prepar pitch first script get start next cours design tap creativ base activ learn actual learn take place within activ write learn link trailer cours view trailer pleas copi past link browser httpsvimeocom382067900b78b800dc0 learner review love approach professor wheeler take toward cours point easi follow inform would definit recommend anyon interest take screenplay write cours cours curriculum simpl adopt profession writer room process write post work peer review share feedback peer revis work feedback receiv peer that real world feel profession writer room yet prior experi writer requir im propon experienti learn activ learn lectur short sometim two minut

In [16]:
cv=CountVectorizer()

In [17]:
vectors=cv.fit_transform(new_data['tags']).toarray()

In [18]:
similarity=cosine_similarity(vectors)

In [19]:
display(new_data)

,Course Name,tags
0,Write A Feature Length Screenplay For Film Or ...,write featur length screenplay film televis be...
1,Business Strategy Business Model Canvas Analys...,busi strategi busi model canva analysi miro be...
2,Silicon Thin Film Solar Cells,silicon thin film solar cell advanc cours cons...
3,Finance Managers,financ manag intermedi come number alway meet ...
4,Retrieve Data using SingleTable SQL Queries,retriev data use singlet sql queri beginn cour...
...,...,...
3517,Capstone Retrieving Processing Visualizing D...,capston retriev process visual data python beg...
3518,Patrick Henry Forgotten Founder,patrick henri forgotten founder intermedi 'giv...
3519,Business intelligence data analytics Generate ...,busi intellig data analyt gener insight advanc...
3520,Rigid Body Dynamics,rigid bodi dynam beginn cours teach dynam one ...


In [20]:
def recommend(course):
  course_index=new_data[new_data['Course Name']==course].first_valid_index()
  distances=similarity[course_index]
  dict={}
  for i in range(0,len(new_data['tags'])):
    dict[i]=distances[i]
  dict=sorted(dict.items(),key=lambda x:x[1], reverse=True)
  for i in range(1,7):
    index=dict[i][0]
    print(new_data['Course Name'][index])

In [23]:
recommend('Retrieve Data using SingleTable SQL Queries')

Creating Database Tables SQL
Advanced SQL Retrieval Queries SQLiteStudio
Create Relational Database Tables Using SQLiteStudio
Manipulating Data SQL
Databases SQL Data Science
Databases SQL Data Science


In [24]:
tfidf=TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(data['tags'])

In [25]:
similarity=cosine_similarity(tfidf_matrix)

In [26]:
recommend('Retrieve Data using SingleTable SQL Queries')

Creating Database Tables SQL
Manipulating Data SQL
Advanced Relational Database SQL
Create Relational Database Tables Using SQLiteStudio
Intermediate Relational Database SQL
Advanced SQL Retrieval Queries SQLiteStudio
